In [1]:
# prep

library(tidyverse)
ess18 <- read_csv("https://github.com/CALDISS-AAU/workshop_R-intro/raw/master/data/ESS2018DK_subset.csv") %>%
    mutate(age = 2018 - yrbrn)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.1.0     v forcats 0.5.1

Warning message:
"pakke 'tibble' blev bygget under R version 4.1.2"
Warning message:
"pakke 'readr' blev bygget under R version 4.1.2"
Warning message:
"pakke 'purrr' blev bygget under R version 4.1.2"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Rows: 1285 Columns: 16

-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (6): vote, prtvtddk, lvpntyr, tygrtr, gndr, edlvddk
dbl (10): idno, netustm, ppltrst, yrbrn, eduyrs, wkhct, wkhtot, grspnum, frl...


i Use `spec()` to retrieve the fu

## Statistical models

There are a lot of packages for creating statistical and there are packages for all kinds of specific analysis.

A recurring element of a lot of these packages and functions however is to specify the model as a function.

Formulas are specified as:
- `y ~ x1 (+x2 +x3 ... +xn)`

### Linear models

Linear models are specified using `lm`.

The code below creates a linear model:

In [2]:
#Linear model for weight and yrbrn
lm(netustm ~ age, data = ess18)


Call:
lm(formula = netustm ~ age, data = ess18)

Coefficients:
(Intercept)          age  
    450.589       -4.559  


In [3]:
#Multiple
lm(netustm ~ age + gndr, data = ess18)


Call:
lm(formula = netustm ~ age + gndr, data = ess18)

Coefficients:
(Intercept)          age     gndrMale  
    446.783       -4.560        7.399  


An advantage of R is the ability to store the model as any other object making it easy to store and recall past results.

In [4]:
#Storing model
lm_model <- lm(netustm ~ age + gndr, data = ess18)

In [5]:
#Summary statistics for bmi_model
summary(lm_model)


Call:
lm(formula = netustm ~ age + gndr, data = ess18)

Residuals:
    Min      1Q  Median      3Q     Max 
-296.31 -117.18  -49.00   76.64  782.84 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 446.7826    16.4553  27.151   <2e-16 ***
age          -4.5597     0.2991 -15.244   <2e-16 ***
gndrMale      7.3987    10.4748   0.706     0.48    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 176.3 on 1131 degrees of freedom
  (151 observations deleted due to missingness)
Multiple R-squared:  0.1707,	Adjusted R-squared:  0.1693 
F-statistic: 116.4 on 2 and 1131 DF,  p-value: < 2.2e-16


### Factors and statistical models

When working with categoricals/factors in R, almost everything about how to treat that categorical in a model should be specified *before* creating the model.

- Should the variable be treated as ordered (nominal) or unordered (ordinal)?
- What value should be used as reference/base?
- Is the ordinal variable to be used as an interval variable?

#### Unordered factors
R will usually coerce character variables to a factor and treat it as nominally scaled (unordered).

To control the reference group, use the `relevel()` function:

In [6]:
ess18 <- ess18 %>%
mutate(gndr = relevel(as.factor(gndr), ref = "Male"))

lm_model <- lm(netustm ~ age + gndr, data = ess18)
summary(lm_model)


Call:
lm(formula = netustm ~ age + gndr, data = ess18)

Residuals:
    Min      1Q  Median      3Q     Max 
-296.31 -117.18  -49.00   76.64  782.84 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 454.1813    16.3585  27.764   <2e-16 ***
age          -4.5597     0.2991 -15.244   <2e-16 ***
gndrFemale   -7.3987    10.4748  -0.706     0.48    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 176.3 on 1131 degrees of freedom
  (151 observations deleted due to missingness)
Multiple R-squared:  0.1707,	Adjusted R-squared:  0.1693 
F-statistic: 116.4 on 2 and 1131 DF,  p-value: < 2.2e-16


#### Ordered factors

When specifying a model with an ordered factor as an independent variable, R will test for different trends (linear, quadratic and cubic) (see https://data.library.virginia.edu/understanding-ordered-factors-in-a-linear-model/ for more details).

In [7]:
ess18 <- ess18 %>%
    mutate(wkhct_cat = case_when(
            wkhct == 37 ~ "37 hours",
            wkhct < 37 ~ "Less than 37 hours",
            wkhct > 37 ~ "More than 37 hours",
            TRUE ~ NA_character_ # specifies the type of missing (character missing)
        ),
           wkhct_cat = factor(wkhct_cat, levels = c('Less than 37 hours', '37 hours', 'More than 37 hours'), 
                              ordered = TRUE))

In [8]:
lm_model <- lm(netustm ~ wkhct_cat, data = ess18)
summary(lm_model)


Call:
lm(formula = netustm ~ wkhct_cat, data = ess18)

Residuals:
    Min      1Q  Median      3Q     Max 
-227.83 -147.83  -72.03   87.31  797.97 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  223.701      7.741  28.900   <2e-16 ***
wkhct_cat.L  -18.801     15.572  -1.207    0.228    
wkhct_cat.Q    2.048     10.817   0.189    0.850    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 192.1 on 1071 degrees of freedom
  (211 observations deleted due to missingness)
Multiple R-squared:  0.001852,	Adjusted R-squared:  -1.173e-05 
F-statistic: 0.9937 on 2 and 1071 DF,  p-value: 0.3705


## Modelling interactions and quadratic terms

**Interactions**

Interactions can be modelled using `*` or `:`:

In [9]:
lm_model <- lm(netustm ~ age + gndr + wkhtot + wkhtot*age, data = ess18)

summary(lm_model)


Call:
lm(formula = netustm ~ age + gndr + wkhtot + wkhtot * age, data = ess18)

Residuals:
    Min      1Q  Median      3Q     Max 
-297.65 -119.43  -47.92   78.82  756.73 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 495.73849   40.05048  12.378  < 2e-16 ***
age          -6.28302    0.86559  -7.259 7.33e-13 ***
gndrFemale   -2.47787   10.82504  -0.229   0.8190    
wkhtot       -1.19896    1.12168  -1.069   0.2853    
age:wkhtot    0.04594    0.02329   1.972   0.0488 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 176.5 on 1110 degrees of freedom
  (170 observations deleted due to missingness)
Multiple R-squared:  0.1736,	Adjusted R-squared:  0.1707 
F-statistic: 58.31 on 4 and 1110 DF,  p-value: < 2.2e-16


**Quadratic terms**

Unfortunately there is no shorthand for doing quadratic terms (at least not with the `lm()` function). 

A variable for the quadratic term has to be created before creating the model:

In [10]:
ess18$quad_wkhtot <- ess18$wkhtot^2

lm_model <- lm(netustm ~ age + gndr + quad_wkhtot, data = ess18)

summary(lm_model)


Call:
lm(formula = netustm ~ age + gndr + quad_wkhtot, data = ess18)

Residuals:
    Min      1Q  Median      3Q     Max 
-299.51 -118.60  -48.33   78.61  768.94 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 438.573990  18.410444  23.822   <2e-16 ***
age          -4.686550   0.310239 -15.106   <2e-16 ***
gndrFemale   -2.879974  10.855392  -0.265   0.7908    
quad_wkhtot   0.013494   0.006805   1.983   0.0476 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 176.7 on 1111 degrees of freedom
  (170 observations deleted due to missingness)
Multiple R-squared:  0.1716,	Adjusted R-squared:  0.1693 
F-statistic:  76.7 on 3 and 1111 DF,  p-value: < 2.2e-16


## (Binomial) logistic regression

Binomial logistic regressions are modelled using the `glm` function. The function is for modelling generalized linear models. To specify a binomial logistic model, one has to specify whether the `family` argument.

In [11]:
ess18 <- ess18 %>%
    mutate(vote_dum = case_when(
        vote == "Yes" ~ 1,
        vote == "No" ~ 0, 
        TRUE ~ as.numeric(NA)
        )
           )

log_model <- glm(vote_dum ~ netustm + age + gndr, data = ess18, family = binomial)
summary(log_model)


Call:
glm(formula = vote_dum ~ netustm + age + gndr, family = binomial, 
    data = ess18)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.1389   0.1642   0.2359   0.3298   0.6724  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) 0.1416315  0.5674770   0.250  0.80291    
netustm     0.0009326  0.0008477   1.100  0.27127    
age         0.0533665  0.0109077   4.893 9.95e-07 ***
gndrFemale  0.8513919  0.3293590   2.585  0.00974 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 384.50  on 1056  degrees of freedom
Residual deviance: 349.66  on 1053  degrees of freedom
  (228 observations deleted due to missingness)
AIC: 357.66

Number of Fisher Scoring iterations: 6
